In [1]:
%matplotlib
# to add the spiking model codebase to the path
import sys, os
sys.path.append('..')
from datetime import datetime

from results import Results
from analysis import (generate_description, FR_weights, determine_first_convergent_trial, determine_drift_rates,
                      FR_scatter, FR_hist2d, FR_hist1d_compare, FR_weights_std, weights_scatter_matrix,
                      weights_hist1d_compare, weights_matrix_compare, combine_two_sessions, calculate_convergence_index)
from seed_files import (crossHomeoPreScalar, crossHomeoPreScalarHomeo, crossHomeoPreScalarHomeoRemean, crossHomeoPreOuterHomeo,
                        crossHomeoPreOuter)
from generate import adjacency_matrix_from_flat_inds
from stats import moving_average

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from brian2 import pA, second, Hz

Using matplotlib backend: Qt5Agg


INFO       Cache size for target "cython": 15322 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\mikejseay\.cython\brian_extensions" directory. [brian2]


In [117]:
basePath = 'C:/Users/mikejseay/Documents/BrianResults/'
movAvgWidth = 31
analysisDicts = (crossHomeoPreScalar, crossHomeoPreScalarHomeo, crossHomeoPreOuter, crossHomeoPreOuterHomeo)
# useSeeds = list(range(9))
# useSeeds = [0, 1, 3, 4, 5, 6, 7, 8]
useSeeds = [0, 1, 2, 3, 4, 5, 6, 7, 8]
allSeries = []

nRules = 4
nSeeds = 9
nTrials = 13530

trialwEEAll = np.empty((nRules, nSeeds, nTrials))
trialwIEAll = np.empty((nRules, nSeeds, nTrials))
trialwEIAll = np.empty((nRules, nSeeds, nTrials))
trialwIIAll = np.empty((nRules, nSeeds, nTrials))
MSEExcUnitsHatAll = np.empty((nRules, nSeeds, nTrials))
MSEInhUnitsHatAll = np.empty((nRules, nSeeds, nTrials))
MSEExcAll = np.empty((nRules, nSeeds, nTrials))
MSEInhAll = np.empty((nRules, nSeeds, nTrials))
distanceFromSetAll = np.empty((nRules, nSeeds, nTrials))
distanceFromSetUnitsAll = np.empty((nRules, nSeeds, nTrials))
frSumExcAll = np.empty((nRules, nSeeds, nTrials - movAvgWidth + 1))
frSumInhAll = np.empty((nRules, nSeeds, nTrials - movAvgWidth + 1))
wSumAll = np.empty((nRules, nSeeds, nTrials - movAvgWidth))
firstConvergentIndAll = np.empty((nRules, nSeeds))
terminalInd = -2000

for analysisInd, analysisDict in enumerate(analysisDicts):
    targetPath = basePath + analysisDict['subfolder']
    for useSeed in useSeeds:
        targetFile1, targetFile2 = analysisDict[useSeed]
        R1 = Results()
        R1.init_from_file(targetFile1, targetPath)
        R2 = Results()
        R2.init_from_file(targetFile2, targetPath)
        R = combine_two_sessions(R1, R2)
        firstConvergentInd = determine_first_convergent_trial(R)
        
        # preparatory calculations
        MSEExc = (R.trialUpFRExc - R.p['setUpFRExc'] / Hz) ** 2
        MSEInh = (R.trialUpFRInh - R.p['setUpFRInh'] / Hz) ** 2
        distanceFromSet = np.sqrt((R.trialUpFRExc - R.p['setUpFRExc'] / Hz) ** 2 + (R.trialUpFRInh - R.p['setUpFRInh'] / Hz) ** 2)
        MSEExcUnits = (R.trialUpFRExcUnits - R.p['setUpFRExc'] / Hz) ** 2
        MSEExcUnitsHat = MSEExcUnits.mean(1)
        MSEInhUnits = (R.trialUpFRInhUnits - R.p['setUpFRInh'] / Hz) ** 2
        MSEInhUnitsHat = MSEInhUnits.mean(1)
        MSEUnitsHat = (MSEExcUnitsHat + MSEInhUnitsHat) / 2
        distanceFromSetUnits = np.sqrt(MSEExcUnits.sum(1) + MSEInhUnits.sum(1))
        trialdwEE = np.ediff1d(R.trialwEE)
        trialdwIE = np.ediff1d(R.trialwIE)
        trialdwEI = np.ediff1d(R.trialwEI)
        trialdwII = np.ediff1d(R.trialwII)
        allAbsdW = np.fabs(trialdwEE) + np.fabs(trialdwIE) + np.fabs(trialdwEI) + np.fabs(trialdwII)
        wSum = calculate_convergence_index(R)
        frSum = moving_average(MSEExcUnits.mean(1), movAvgWidth) + moving_average(MSEInhUnits.mean(1), movAvgWidth)
        movAvgWidth=31
        frSumExc = moving_average(MSEExcUnits.mean(1), movAvgWidth)
        frSumInh = moving_average(MSEInhUnits.mean(1), movAvgWidth)
        frSum = frSumExc + frSumInh
        
        # scalar quantifiers
        meanAbsdwEE = np.fabs(trialdwEE).mean()
        meanAbsdwIE = np.fabs(trialdwIE).mean()
        meanAbsdwEI = np.fabs(trialdwEI).mean()
        meanAbsdwII = np.fabs(trialdwII).mean()
        meanAbsdwSum = meanAbsdwEE + meanAbsdwIE + meanAbsdwEI + meanAbsdwII
        meanSumAbsMAvgDiffdW = wSum.mean()
        meanSumAbsMAvgDiffTerminaldW = wSum[terminalInd:].mean()
        meanSumAbsMAvgDiffFRMSE = frSum.mean()
        meanSumAbsMAvgDiffTerminalFRMSE = frSum[terminalInd:].mean()
        meanAbsTerminaldwEE = np.fabs(trialdwEE[terminalInd:]).mean()
        meanAbsTerminaldwIE = np.fabs(trialdwIE[terminalInd:]).mean()
        meanAbsTerminaldwEI = np.fabs(trialdwEI[terminalInd:]).mean()
        meanAbsTerminaldwII = np.fabs(trialdwII[terminalInd:]).mean()
        meanAbsTerminaldwSum = meanAbsTerminaldwEE + meanAbsTerminaldwIE + meanAbsTerminaldwEI + meanAbsTerminaldwII
        
        meanDistanceFromSet = distanceFromSet.mean()
        meanTerminalDistanceFromSet = distanceFromSet[terminalInd:].mean()
        meanDistanceFromSetUnits = distanceFromSetUnits.mean()
        meanTerminalDistanceFromSetUnits = distanceFromSetUnits[terminalInd:].mean()
        
        minMSEExc = MSEExc.min()
        meanMSEExc = MSEExc.mean()
        meanTerminalMSEExc = MSEExc[terminalInd:].mean()
        minMSEExcUnits = MSEExcUnitsHat.min()
        meanMSEExcUnits = MSEExcUnitsHat.mean()
        meanTerminalMSEExcUnits = MSEExcUnitsHat[terminalInd:].mean()
        minMSEInh = MSEInh.min()
        meanMSEInh = MSEInh.mean()
        meanTerminalMSEInh = MSEInh[terminalInd:].mean()
        minMSEInhUnits = MSEInhUnitsHat.min()
        meanMSEInhUnits = MSEInhUnitsHat.mean()
        meanTerminalMSEInhUnits = MSEInhUnitsHat[terminalInd:].mean()
        meanTerminalMSEUnits = MSEUnitsHat[terminalInd:].mean()
        
        MSEExcUnitsHatAll[analysisInd, useSeed, :] = MSEExcUnitsHat
        MSEInhUnitsHatAll[analysisInd, useSeed, :] = MSEInhUnitsHat
        MSEExcAll[analysisInd, useSeed, :] = MSEExc
        MSEInhAll[analysisInd, useSeed, :] = MSEInh
        distanceFromSetAll[analysisInd, useSeed, :] = distanceFromSet
        distanceFromSetUnitsAll[analysisInd, useSeed, :] = distanceFromSetUnits
        frSumExcAll[analysisInd, useSeed, :] = frSumExc
        frSumInhAll[analysisInd, useSeed, :] = frSumInh
        wSumAll[analysisInd, useSeed, :] = wSum
        firstConvergentIndAll[analysisInd, useSeed] = firstConvergentInd
        trialwEEAll[analysisInd, useSeed, :] = R.trialwEE
        trialwIEAll[analysisInd, useSeed, :] = R.trialwIE
        trialwEIAll[analysisInd, useSeed, :] = R.trialwEI
        trialwIIAll[analysisInd, useSeed, :] = R.trialwII
        
        # build a pandas series with all of the above info for the given seed
        scalarQuantVals = (meanDistanceFromSet, meanTerminalDistanceFromSet, meanDistanceFromSetUnits, meanTerminalDistanceFromSetUnits, meanAbsdwEE, meanAbsdwIE, meanAbsdwEI, meanAbsdwII, meanAbsdwSum, meanSumAbsMAvgDiffdW, meanSumAbsMAvgDiffTerminaldW, meanSumAbsMAvgDiffFRMSE, meanSumAbsMAvgDiffTerminalFRMSE, meanAbsTerminaldwEE, meanAbsTerminaldwIE, meanAbsTerminaldwEI, meanAbsTerminaldwII, meanAbsTerminaldwSum, minMSEExc, meanMSEExc, meanTerminalMSEExc, minMSEExcUnits, meanMSEExcUnits, meanTerminalMSEExcUnits, minMSEInh, meanMSEInh, meanTerminalMSEInh, minMSEInhUnits, meanMSEInhUnits, meanTerminalMSEInhUnits, meanTerminalMSEUnits, firstConvergentInd + 1)
        scalarQuantLbls = ('meanDistanceFromSet', 'meanTerminalDistanceFromSet', 'meanDistanceFromSetUnits', 'meanTerminalDistanceFromSetUnits', 'meanAbsdwEE', 'meanAbsdwIE', 'meanAbsdwEI', 'meanAbsdwII', 'meanAbsdwSum', 'meanSumAbsMAvgDiffdW', 'meanSumAbsMAvgDiffTerminaldW', 'meanSumAbsMAvgDiffFRMSE', 'meanSumAbsMAvgDiffTerminalFRMSE', 'meanAbsTerminaldwEE', 'meanAbsTerminaldwIE', 'meanAbsTerminaldwEI', 'meanAbsTerminaldwII', 'meanAbsTerminaldwSum', 'minMSEExc', 'meanMSEExc', 'meanTerminalMSEExc', 'minMSEExcUnits', 'meanMSEExcUnits', 'meanTerminalMSEExcUnits', 'minMSEInh', 'meanMSEInh', 'meanTerminalMSEInh', 'minMSEInhUnits', 'meanMSEInhUnits', 'meanTerminalMSEInhUnits', 'meanTerminalMSEUnits', 'trialsToConverge')
        analysisName = (R.p['useRule'], R.p['rngSeed'])
#         analysisName = (R.p['rngSeed'], R.p['useRule'])

        currentSeries = pd.Series(scalarQuantVals, index=scalarQuantLbls, name=analysisName)
        allSeries.append(currentSeries)

In [118]:
MSEAll = MSEExcAll + MSEInhAll
MSEUnitsHatAll = (MSEExcUnitsHatAll + MSEInhUnitsHatAll) / 2
frSumAll = frSumExcAll + frSumInhAll

In [119]:
totalEToSingleEAll = trialwEEAll * R.p['nExc'] * R.p['propConnect']
totalIToSingleEAll = trialwEIAll * R.p['nInh'] * R.p['propConnect']
totalEToSingleIAll = trialwIEAll * R.p['nExc'] * R.p['propConnect']
totalIToSingleIAll = trialwIIAll * R.p['nInh'] * R.p['propConnect']

netCurrentToSingleEAll = totalEToSingleEAll - totalIToSingleEAll
netCurrentToSingleIAll = totalEToSingleIAll - totalIToSingleIAll

In [120]:
overallDF = pd.concat(allSeries, axis=1).transpose()

ruleRenamer = {
    'cross-homeo-pre-scalar': 'CH-global',
    'cross-homeo-pre-scalar-homeo': '2T-global',
    'cross-homeo-pre-outer': 'CH-local',
    'cross-homeo-pre-outer-homeo': '2T-local',
}

ruleValues = overallDF.index.get_level_values(0).to_list()
ruleValuesRenamed = [ruleRenamer[rV] for rV in ruleValues]

overallDF['rule'] = ruleValuesRenamed

In [121]:
# optionally drop the data from seed = 2 and/or 6

overallDF = overallDF.iloc[overallDF.index.get_level_values(1) != 2, :]
overallDF = overallDF.iloc[overallDF.index.get_level_values(1) != 6, :]

In [122]:
analysisLabels = ('cross-homeo-pre-scalar',
'cross-homeo-pre-scalar-homeo',
'cross-homeo-pre-outer',
'cross-homeo-pre-outer-homeo')

In [69]:
# for each measure, do a violinplot showing all 4 learning rules against each other
# to do this we need a small dataframe with two columns: the measure, and the label
# the dataframe ROWS should have all
# so basically this requires making the label (the learning rule) a column

In [123]:
measureLst = overallDF.columns.to_list()[:-1]

In [9]:
measureLst

['meanDistanceFromSet',
 'meanTerminalDistanceFromSet',
 'meanDistanceFromSetUnits',
 'meanTerminalDistanceFromSetUnits',
 'meanAbsdwEE',
 'meanAbsdwIE',
 'meanAbsdwEI',
 'meanAbsdwII',
 'meanAbsdwSum',
 'meanSumAbsMAvgDiffdW',
 'meanSumAbsMAvgDiffTerminaldW',
 'meanSumAbsMAvgDiffFRMSE',
 'meanSumAbsMAvgDiffTerminalFRMSE',
 'meanAbsTerminaldwEE',
 'meanAbsTerminaldwIE',
 'meanAbsTerminaldwEI',
 'meanAbsTerminaldwII',
 'minMSEExc',
 'meanMSEExc',
 'meanTerminalMSEExc',
 'minMSEExcUnits',
 'meanMSEExcUnits',
 'meanTerminalMSEExcUnits',
 'minMSEInh',
 'meanMSEInh',
 'meanTerminalMSEInh',
 'minMSEInhUnits',
 'meanMSEInhUnits',
 'meanTerminalMSEInhUnits',
 'trialsToConverge']

In [124]:
selectMeasureLst = ('meanSumAbsMAvgDiffTerminalFRMSE', 'meanSumAbsMAvgDiffTerminaldW', 'trialsToConverge',)

In [125]:
plt.close('all')

In [116]:
# plot dedicated to showing the chosen MSE measure

f, ax = plt.subplots(figsize=(5, 6))

measureName = 'meanTerminalMSEUnits'

sns.barplot(x='rule', y=measureName, data=overallDF, ax=ax)
sns.stripplot(x='rule', y=measureName, data=overallDF, color='k', ax=ax)
ax.set(title=measureName)
axr[measureInd].set(ylabel='MSE')
plt.yscale('log')

In [128]:
# plot dedicated to showing the mean MSE, for the average FR and the unit FR

f, ax = plt.subplots(2, 2, sharex=True, sharey=False, figsize=(10, 10))
axr = ax.ravel()

# weightMeasures = ('meanMSEExc', 'meanMSEInh', 'meanMSEExcUnits','meanMSEInhUnits',)
weightMeasures = ('meanTerminalMSEExc', 'meanTerminalMSEInh', 'meanTerminalMSEExcUnits', 'meanTerminalMSEInhUnits',)

for measureInd, measureName in enumerate(weightMeasures):
    sns.barplot(x='rule', y=measureName, data=overallDF, ax=axr[measureInd])
#     sns.boxplot(x='rule', y=measureName, data=overallDF, width=.6, showfliers=False)
    sns.stripplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
#     sns.swarmplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
    axr[measureInd].set(title=measureName)
    if measureInd <= 1:
        axr[measureInd].set(xlabel='')
    if (measureInd % 2) == 1:
        axr[measureInd].set(ylabel='')
    else:
        axr[measureInd].set(ylabel='MSE')

In [25]:
from statsmodels import stats

In [28]:
dir(stats)

['PytestTester',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_adnorm',
 '_knockoff',
 '_lilliefors',
 '_lilliefors_critical_values',
 'anova',
 'api',
 'base',
 'contingency_tables',
 'contrast',
 'correlation_tools',
 'descriptivestats',
 'diagnostic',
 'gof',
 'mediation',
 'moment_helpers',
 'multicomp',
 'multitest',
 'oaxaca',
 'power',
 'proportion',
 'sandwich_covariance',
 'stattools',
 'tabledist',
 'test',
 'weightstats']

In [20]:
import statsmodels

In [21]:
statsmodels.__version__

'0.12.2'

In [74]:
# plot dedicated to showing the euclidean distance measures

f, ax = plt.subplots(2, 2, sharex=True, sharey=False, figsize=(10, 10))
axr = ax.ravel()

weightMeasures = ('meanDistanceFromSet', 'meanTerminalDistanceFromSet', 'meanDistanceFromSetUnits', 'meanTerminalDistanceFromSetUnits',)

for measureInd, measureName in enumerate(weightMeasures):
    sns.barplot(x='rule', y=measureName, data=overallDF, ax=axr[measureInd])
#     sns.boxplot(x='rule', y=measureName, data=overallDF, width=.6, showfliers=False)
    sns.stripplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
#     sns.swarmplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
    axr[measureInd].set(title=measureName)
    if measureInd <= 1:
        axr[measureInd].set(xlabel='')
    if (measureInd % 2) == 1:
        axr[measureInd].set(ylabel='')
    else:
        axr[measureInd].set(ylabel='Euclidean distance')

In [126]:
# plot dedicated to showing the chosen MACW measure

f, ax = plt.subplots(figsize=(5, 6))

measureName = 'meanAbsTerminaldwSum'

sns.barplot(x='rule', y=measureName, data=overallDF, ax=ax)
sns.stripplot(x='rule', y=measureName, data=overallDF, color='k', ax=ax)
ax.set(title=measureName)
axr[measureInd].set(ylabel='MACW')
plt.yscale('log')

In [129]:
# plot dedicated to showing the mean(abs(dW)) for all 4 weight classes
f, ax = plt.subplots(2, 2, sharex=True, sharey=True, figsize=(10, 10))
axr = ax.ravel()

# weightMeasures = ('meanAbsdwEE', 'meanAbsdwIE', 'meanAbsdwEI', 'meanAbsdwII',)
weightMeasures = ('meanAbsTerminaldwEE', 'meanAbsTerminaldwIE', 'meanAbsTerminaldwEI', 'meanAbsTerminaldwII',)

for measureInd, measureName in enumerate(weightMeasures):
    sns.barplot(x='rule', y=measureName, data=overallDF, ax=axr[measureInd])
#     sns.boxplot(x='rule', y=measureName, data=overallDF, width=.6, showfliers=False)
    sns.stripplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
#     sns.swarmplot(x='rule', y=measureName, data=overallDF, color='k', ax=axr[measureInd])
    axr[measureInd].set(title=measureName)
    if measureInd <= 1:
        axr[measureInd].set(xlabel='')
    if (measureInd % 2) == 1:
        axr[measureInd].set(ylabel='')
    else:
        axr[measureInd].set(ylabel='mean(abs(dW))')

axr[measureInd].set(ylim=(0, 0.3))

[(0.0, 0.3)]

In [76]:
# the rest

for measureInd, measureName in enumerate(selectMeasureLst):
    f, ax = plt.subplots()
    sns.barplot(x='rule', y=measureName, data=overallDF)
#     sns.boxplot(x='rule', y=measureName, data=overallDF, width=.6, showfliers=False)
#     sns.stripplot(x='rule', y=measureName, data=overallDF, color='k')
    sns.swarmplot(x='rule', y=measureName, data=overallDF, color='k')

# NON BAR GRAPHS

In [131]:
# plot that shows convergence of all 4 learning rules on one graph for one seed

useData = wSumAll  # frSumAll, wSumAll, MSEAll, distanceFromSetAll, distanceFromSetUnitsAll
useMovAvg = True
useLogYScale = False
movAvgWidth = 31
useYLabel = 'MACW'

f, ax = plt.subplots()

useSeed = 8
for analysisInd, analysisLabel in enumerate(analysisLabels):
    useFirstConvergentInd = int(firstConvergentIndAll[analysisInd, useSeed])
    yData = useData[analysisInd, useSeed, :]
    if useMovAvg:
        yData = moving_average(yData, movAvgWidth)
    ax.plot(yData, label=ruleRenamer[analysisLabel], alpha=0.7)
if useLogYScale:
    plt.yscale('log')
ax.legend()
ax.set(xlabel='Trial #', ylabel=useYLabel)

[Text(0.5, 0, 'Trial #'), Text(0, 0.5, 'MACW')]

In [130]:
# plot that shows convergence of all 4 learning rules on one graph for each seed

useData = wSumAll  # frSumAll, wSumAll, MSEAll, distanceFromSetAll, distanceFromSetUnitsAll
useMovAvg = True
useLogYScale = False
movAvgWidth = 31
useYLabel = 'MACW'
# useYLims = (1e1, useData.max())
useYLims = (0, 1)

f, ax = plt.subplots(3, 3, sharex=True, sharey=True)
axr = ax.ravel()

for useSeed in useSeeds:
    for analysisInd, analysisLabel in enumerate(analysisLabels):
        useFirstConvergentInd = int(firstConvergentIndAll[analysisInd, useSeed])
        yData = useData[analysisInd, useSeed, :]
        if useMovAvg:
            yData = moving_average(yData, movAvgWidth)
        axr[useSeed].plot(yData, label=ruleRenamer[analysisLabel], alpha=0.5)
    if useLogYScale:
        plt.yscale('log')
    axr[useSeed].set(title=str(useSeed), ylim=useYLims)
    if useSeed == 2:
        axr[useSeed].legend()
    if useSeed % 3 == 0:
        axr[useSeed].set(ylabel=useYLabel)
    if useSeed >= 6:
        axr[useSeed].set(xlabel='Trial #')

In [80]:
# plot that shows convergence of all 9 seeds on one graph for only one learning rule

useData = frSumAll  # frSumAll, wSumAll, MSEAll

f, ax = plt.subplots(2, 2, sharex=True, sharey=True)
axr = ax.ravel()

useSeedsLocal = (0, 1, 3, 4, 5, 7, 8)

for analysisInd, analysisLabel in enumerate(analysisLabels):
    for useSeed in useSeedsLocal:
        axr[analysisInd].plot(useData[analysisInd, useSeed, :], label=str(useSeed), alpha=.5)
    plt.yscale('log')
    axr[analysisInd].set(title=ruleRenamer[analysisLabel], ylim=(1e1, useData.max()))
    if analysisInd == 2:
        axr[analysisInd].legend()

In [31]:
# original way, net current

totalEToSingleEAll = trialwEEAll * R.p['nExc'] * R.p['propConnect']
totalIToSingleEAll = trialwEIAll * R.p['nInh'] * R.p['propConnect']
totalEToSingleIAll = trialwIEAll * R.p['nExc'] * R.p['propConnect']
totalIToSingleIAll = trialwIIAll * R.p['nInh'] * R.p['propConnect']

# make the numbers smaller for crybaby Dean

netCurrentToSingleEAll = totalEToSingleEAll - totalIToSingleEAll
netCurrentToSingleIAll = totalEToSingleIAll - totalIToSingleIAll

# netCurrentToSingleEAll = (totalEToSingleEAll - totalIToSingleEAll) / R.p['nUnits'] * R.p['propConnect']
# netCurrentToSingleIAll = (totalEToSingleIAll - totalIToSingleIAll) / R.p['nUnits'] * R.p['propConnect']

In [25]:
# different way, weights

netCurrentToSingleEAll = trialwEEAll - trialwEIAll
netCurrentToSingleIAll = trialwIEAll - trialwIIAll

In [81]:
# plot that shows INITIAL CONDITIONS of all 9 seeds on one graph for only one learning rule, but in a 2D system defined by
# the net current to each E unit (x) vs. the net current to each I unit (y)

useSeedsLocal = (0, 1, 3, 4, 5, 7, 8,)
# useSeedsLocal = (0, 1, 2, 3, 4, 5, 6, 7, 8)
useColors = ('magenta', 'red', 'grey', 'orange', 'green', 'blue', 'purple', 'cyan', 'black')
useAnalysisInd = 3

f, ax = plt.subplots(figsize=(8, 5))
for useSeed in useSeedsLocal:
    ax.scatter(netCurrentToSingleEAll[useAnalysisInd, useSeed, 0], netCurrentToSingleIAll[useAnalysisInd, useSeed, 0], 30,
               color=useColors[useSeed], marker='^')

ax.set(xlabel='Net weight onto each E unit (on average)', ylabel='Net weight onto each I unit (on average)')

[Text(0.5, 0, 'Net weight onto each E unit (on average)'),
 Text(0, 0.5, 'Net weight onto each I unit (on average)')]

In [35]:
ax.get_ylim()

(-102468.76635394698, 155546.3496547005)

In [47]:
from stats import regress_linear

In [99]:
plt.close('all')

In [63]:
usewEEAll = trialwEEAll * R.p['nExc'] * R.p['propConnect']
usewEIAll = trialwEIAll * R.p['nInh'] * R.p['propConnect']
usewIEAll = trialwIEAll * R.p['nExc'] * R.p['propConnect']
usewIIAll = trialwIIAll * R.p['nInh'] * R.p['propConnect']

In [82]:
# plot that shows convergence of all 9 seeds on one graph for only one learning rule, but in a 2D system defined by
# the net current to each E unit (x) vs. the net current to each I unit (y)

useSeedsLocal = (0, 1, 3, 4, 5, 7, 8,)
# useSeedsLocal = (0, 1, 2, 3, 4, 5, 6, 7, 8)
useColors = ('magenta', 'red', 'grey', 'orange', 'green', 'blue', 'purple', 'cyan', 'black')
useAnalysisInd = 3

xVals = usewEEAll[useAnalysisInd, useSeedsLocal, -1]
yVals = usewEIAll[useAnalysisInd, useSeedsLocal, -1]

pred_x, pred_y, b, r2, p = regress_linear(xVals, yVals)

f, ax = plt.subplots(2, 1, figsize=(6, 10))

ax[0].plot(pred_x, pred_y, color='k')
for useSeed in useSeedsLocal:
    ax[0].plot(usewEEAll[useAnalysisInd, useSeed, :], usewEIAll[useAnalysisInd, useSeed, :],
            label=str(useSeed), alpha=.5, color=useColors[useSeed])
    ax[0].scatter(usewEEAll[useAnalysisInd, useSeed, 0], usewEIAll[useAnalysisInd, useSeed, 0], 30,
               color=useColors[useSeed], marker='^')
    ax[0].scatter(usewEEAll[useAnalysisInd, useSeed, -1], usewEIAll[useAnalysisInd, useSeed, -1], 30,
               color=useColors[useSeed], marker='v')

ax[0].axes.xaxis.set_ticks([])
ax[0].axes.yaxis.set_ticks([])

ax[0].set(xlabel='wEE', ylabel='wEI')
ax[0].legend()

xVals = usewIEAll[useAnalysisInd, useSeedsLocal, -1]
yVals = usewIIAll[useAnalysisInd, useSeedsLocal, -1]

pred_x, pred_y, b, r2, p = regress_linear(xVals, yVals)

ax[1].plot(pred_x, pred_y, color='k')
for useSeed in useSeedsLocal:
    ax[1].plot(usewIEAll[useAnalysisInd, useSeed, :], usewIIAll[useAnalysisInd, useSeed, :],
            label=str(useSeed), alpha=.5, color=useColors[useSeed])
    ax[1].scatter(usewIEAll[useAnalysisInd, useSeed, 0], usewIIAll[useAnalysisInd, useSeed, 0], 30,
               color=useColors[useSeed], marker='^')
    ax[1].scatter(usewIEAll[useAnalysisInd, useSeed, -1], usewIIAll[useAnalysisInd, useSeed, -1], 30,
               color=useColors[useSeed], marker='v')

ax[1].axes.xaxis.set_ticks([])
ax[1].axes.yaxis.set_ticks([])

ax[1].set(xlabel='wIE', ylabel='wII')
ax[1].legend()

In [51]:
# plot that shows convergence of all 9 seeds on one graph for only one learning rule, but in a 2D system defined by
# the wEE and wEI

useSeedsLocal = (0, 1, 3, 4, 5, 7, 8,)
# useSeedsLocal = (0, 1, 2 3, 4, 5, 6, 7, 8)
useColors = ('magenta', 'red', 'grey', 'orange', 'green', 'blue', 'purple', 'cyan', 'black')
useAnalysisInd = 3

xVals = netCurrentToSingleEAll[useAnalysisInd, useSeedsLocal, -1]
yVals = netCurrentToSingleIAll[useAnalysisInd, useSeedsLocal, -1]

pred_x, pred_y, b, r2, p = regress_linear(xVals, yVals)

f, ax = plt.subplots(figsize=(9, 7))
ax.plot(pred_x, pred_y, color='k')
for useSeed in useSeedsLocal:
    ax.plot(netCurrentToSingleEAll[useAnalysisInd, useSeed, :], netCurrentToSingleIAll[useAnalysisInd, useSeed, :],
            label=str(useSeed), alpha=.5, color=useColors[useSeed])
    ax.scatter(netCurrentToSingleEAll[useAnalysisInd, useSeed, 0], netCurrentToSingleIAll[useAnalysisInd, useSeed, 0], 30,
               color=useColors[useSeed], marker='^')
    ax.scatter(netCurrentToSingleEAll[useAnalysisInd, useSeed, -1], netCurrentToSingleIAll[useAnalysisInd, useSeed, -1], 30,
               color=useColors[useSeed], marker='v')
ax.hlines(0, *ax.get_xlim(), color='k', linestyle='--', alpha=0.2)
ax.vlines(0, *ax.get_ylim(), color='k', linestyle='--', alpha=0.2)

ax.axes.xaxis.set_ticks([])
ax.axes.yaxis.set_ticks([])

ax.set(xlabel='Mean synaptic current onto E units', ylabel='Mean synaptic current onto I units')
ax.legend()

In [ ]:
# additional measure: # of trials to convergence (double check!!!)

In [ ]:
# additional measures: pre-convergent values

In [28]:
overallDF.to_csv(os.getcwd() + '\\seedResults.csv')

In [30]:
overallDF.groupby(level=0).mean()

,meanAbsdwEE,meanAbsdwIE,meanAbsdwEI,meanAbsdwII,meanSumAbsMAvgDiffdW,meanSumAbsMAvgDiffConvergentdW,meanSumAbsMAvgDiffFRMSE,meanSumAbsMAvgDiffConvergentFRMSE,meanAbsPostConvergencedwEE,meanAbsPostConvergencedwIE,...,meanConvergentMSEExc,minMSEExcUnits,meanMSEExcUnits,meanConvergentMSEExcUnits,minMSEInh,meanMSEInh,meanConvergentMSEInh,minMSEInhUnits,meanMSEInhUnits,meanConvergentMSEInhUnits
0,0.057160,0.029620,0.194129,0.074381,0.141574,0.103030,2823.970796,2856.579789,0.044826,0.022205,...,8.509668,9.951347,232.634200,227.347556,0.000021,71.385862,67.700631,57.122678,2590.166666,2628.103358
1,0.094674,0.047954,0.290633,0.118944,0.185059,0.152298,3951.738314,3526.447666,0.081022,0.039941,...,191.302233,8.044849,872.539134,440.432213,0.000021,280.511193,197.972814,66.175572,3228.761297,3091.551480
2,0.154092,0.076263,0.428524,0.183083,0.255322,0.210957,16640.069687,16121.163998,0.147020,0.070511,...,2301.916613,10.791591,4343.776532,3029.728984,0.003185,4117.237460,4802.839801,82.643704,12545.081731,13084.209525
3,0.033611,0.013789,0.124078,0.033865,0.110457,0.079476,7826.012197,8199.675330,0.025655,0.009619,...,9.781097,6.746211,679.843213,709.225561,0.000977,380.937963,389.636368,63.363649,7140.821404,7484.347565
4,0.042073,0.020846,0.157902,0.056681,0.104929,0.077250,8299.170825,9592.480124,0.030489,0.015502,...,13.029149,7.705842,734.867557,834.033462,0.183035,412.555571,445.185071,59.654409,7558.122799,8758.260762
5,0.064156,0.033057,0.221891,0.084894,0.154651,0.115868,4484.105405,3245.788742,0.045674,0.022842,...,4.368634,10.515587,927.764743,236.717316,0.000016,671.887128,101.962647,65.549266,3967.428731,3006.860887
6,0.042064,0.020458,0.130534,0.044326,0.190723,0.088904,2733.131121,2848.930939,0.017761,0.008040,...,1.243542,7.247181,200.201422,210.191570,0.002198,99.580197,40.532338,54.977552,2531.829666,2637.297945
7,0.025470,0.011906,0.111210,0.030974,0.124954,0.086969,2814.244710,2868.801752,0.016693,0.007354,...,1.534841,6.628933,197.644560,201.055298,0.013254,61.488243,51.211928,55.811700,2614.738836,2665.845816
8,0.032891,0.016210,0.144658,0.048043,0.092354,0.082147,3269.479101,3265.773337,0.018979,0.008631,...,8.717169,9.830707,244.676393,239.876854,0.000070,175.371213,160.482102,60.913485,3044.370333,3024.113688


In [12]:
test

In [103]:
basePath = 'C:/Users/mikejseay/Documents/BrianResults/'

# analysisDict = crossHomeoPreScalar.copy()
analysisDict = crossHomeoPreScalarHomeo.copy()
# analysisDict = crossHomeoPreOuter.copy()
# analysisDict = crossHomeoPreOuterHomeo.copy()
# analysisDict = crossHomeoPreScalarHomeoRemean.copy()

useSeed = 2

In [104]:
targetPath = basePath + analysisDict['subfolder']

In [105]:
targetFile1, targetFile2 = analysisDict[useSeed]
R1 = Results()
R1.init_from_file(targetFile1, targetPath)
R2 = Results()
R2.init_from_file(targetFile2, targetPath)
R = combine_two_sessions(R1, R2)
print(R.rID)

classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_seed2__2021-08-21-09-34


In [106]:
print(R.p['setUpFRExc'], R.p['tauUpFRTrials'], R.p['alpha1'], R.p['spikeInputAmplitude'])
plt.close('all')
pdfObject = PdfPages(targetPath + '/reports/' + R.rID + '_report.pdf')
removeOutliers = True

5. Hz 1 8. fC 0.98


In [107]:
generate_description(R)

FR_weights(R)
plt.savefig(pdfObject, format='pdf')

# firstConvergentInd = int(R.trialwEE.size / 2)
# firstConvergentInd = 4700
firstConvergentInd = determine_first_convergent_trial(R)
# firstConvergentInd = determine_first_convergent_trial(R) + 1500
print('We think the first convergent index is', firstConvergentInd)

We think the first convergent index is 5626


In [108]:
determine_drift_rates(R, firstConvergentInd)
print(R.driftRateEE)
print(R.driftRateEI)
print(R.driftRateIE)
print(R.driftRateII)

# convergence process
FR_scatter(R, endTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')
FR_hist1d_compare(R, endTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')
FR_weights_std(R, endTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')
weights_scatter_matrix(R, endTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')

# post convergence
FR_scatter(R, startTrialInd=firstConvergentInd, removeOutliers=removeOutliers)
plt.savefig(pdfObject, format='pdf')
FR_hist2d(R, startTrialInd=firstConvergentInd, removeOutliers=removeOutliers)
plt.savefig(pdfObject, format='pdf')
FR_hist1d_compare(R, startTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')
FR_weights_std(R, startTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')
weights_scatter_matrix(R, startTrialInd=firstConvergentInd)
plt.savefig(pdfObject, format='pdf')

# epilogue
weights_hist1d_compare(R)
plt.savefig(pdfObject, format='pdf')
weights_matrix_compare(R)
plt.savefig(pdfObject, format='pdf')

pdfObject.close()

1.7345976444595907
0.7466571490226588
1.018813379420174
-1.0040977452787898


WARNING    ..\stats.py:17: RuntimeWarning: invalid value encountered in greater
  a_outlying = np.logical_or(a > a_mean + sd_thresh * a_std, a < a_mean - sd_thresh * a_std)
 [py.warnings]
WARNING    ..\stats.py:17: RuntimeWarning: invalid value encountered in less
  a_outlying = np.logical_or(a > a_mean + sd_thresh * a_std, a < a_mean - sd_thresh * a_std)
 [py.warnings]
WARNING    ..\analysis.py:333: RuntimeWarning: invalid value encountered in less
  wFullInitPlot[wFullInitPlot < vlims[0]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:334: RuntimeWarning: invalid value encountered in greater
  wFullInitPlot[wFullInitPlot > vlims[1]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:336: RuntimeWarning: invalid value encountered in less
  wFullFinalPlot[wFullFinalPlot < vlims[0]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:337: RuntimeWarning: invalid value encountered in greater
  wFullFinalPlot[wFullFinalPlot > vlims[1]] = np.nan
 [py.warnings]
WARNING    C:\Users\mikejseay

In [9]:
plt.close('all')

In [10]:
# goals:
# quantify the MSE at each trial
# quantify the mean(abs(dWEE)) at each step..
# create a common plot across multiple seeds

In [144]:
# preparatory calculations

MSEExc = (R.trialUpFRExc - R.p['setUpFRExc'] / Hz) ** 2
MSEInh = (R.trialUpFRInh - R.p['setUpFRInh'] / Hz) ** 2
MSEExcUnits = (R.trialUpFRExcUnits - R.p['setUpFRExc'] / Hz) ** 2
MSEExcUnitsHat = MSEExcUnits.mean(1)
MSEInhUnits = (R.trialUpFRInhUnits - R.p['setUpFRInh'] / Hz) ** 2
MSEInhUnitsHat = MSEInhUnits.mean(1)
trialdwEE = np.ediff1d(R.trialwEE)
trialdwIE = np.ediff1d(R.trialwIE)
trialdwEI = np.ediff1d(R.trialwEI)
trialdwII = np.ediff1d(R.trialwII)
allAbsdW = np.fabs(trialdwEE) + np.fabs(trialdwIE) + np.fabs(trialdwEI) + np.fabs(trialdwII)
wSum = calculate_convergence_index(R)
frSum = moving_average(MSEExcUnits.mean(1), movAvgWidth) + moving_average(MSEInhUnits.mean(1), movAvgWidth)
movAvgWidth=31
frSumExc = moving_average(MSEExcUnits.mean(1), movAvgWidth)
frSumInh = moving_average(MSEInhUnits.mean(1), movAvgWidth)
frSum = frSumExc + frSumInh

In [145]:
# scalar quantifiers

meanAbsdwEE = np.fabs(trialdwEE).mean()
meanAbsdwIE = np.fabs(trialdwIE).mean()
meanAbsdwEI = np.fabs(trialdwEI).mean()
meanAbsdwII = np.fabs(trialdwII).mean()
meanSumAbsMAvgDiffdW = wSum.mean()
meanSumAbsMAvgDiffConvergentdW = wSum[firstConvergentInd:].mean()
meanSumAbsMAvgDiffFRMSE = frSum.mean()
meanSumAbsMAvgDiffConvergentFRMSE = frSum[firstConvergentInd:].mean()
meanAbsPostConvergencedwEE = np.fabs(trialdwEE[firstConvergentInd:]).mean()
meanAbsPostConvergencedwIE = np.fabs(trialdwIE[firstConvergentInd:]).mean()
meanAbsPostConvergencedwEI = np.fabs(trialdwEI[firstConvergentInd:]).mean()
meanAbsPostConvergencedwII = np.fabs(trialdwII[firstConvergentInd:]).mean()
minMSEExc = MSEExc.min()
meanMSEExc = MSEExc.mean()
meanConvergentMSEExc = MSEExc[firstConvergentInd:].mean()
minMSEExcUnits = MSEExcUnitsHat.min()
meanMSEExcUnits = MSEExcUnitsHat.mean()
meanConvergentMSEExcUnits = MSEExcUnitsHat[firstConvergentInd:].mean()
minMSEInh = MSEInh.min()
meanMSEInh = MSEInh.mean()
meanConvergentMSEInh = MSEInh[firstConvergentInd:].mean()
minMSEInhUnits = MSEInhUnitsHat.min()
meanMSEInhUnits = MSEInhUnitsHat.mean()
meanConvergentMSEInhUnits = MSEInhUnitsHat[firstConvergentInd:].mean()

In [146]:
plt.close('all')

In [140]:
# graphs

f, ax = plt.subplots()
ax.plot(MSEExcUnitsHat, color='green', linewidth=1)
ax.plot(MSEInhUnitsHat, color='red', linewidth=1)
plt.yscale('log')
ax.legend(('Exc', 'Inh'))
ax.set(xlabel='Trial #', ylabel='MSE for avg. FR (Hz ^ 2)')

[Text(0, 0.5, 'MSE for avg. FR (Hz ^ 2)'), Text(0.5, 0, 'Trial #')]

In [131]:
minVal = 1e-6

zData = np.arange(MSEExc.size) + 2

f, ax = plt.subplots()
ax.scatter(MSEExc, MSEInh, s=3, c=zData, cmap=plt.cm.viridis, alpha=0.5)
plt.xscale('log')
plt.yscale('log')
ax.set(xlim=(minVal, MSEExc.max()), ylim=(minVal, MSEInh.max()),
       xlabel='MSE for avg. E FR (Hz ^ 2)', ylabel='MSE for avg. I FR (Hz ^ 2)')

[(1e-06, 101964.4336322184),
 Text(0, 0.5, 'MSE for avg. I FR (Hz ^ 2)'),
 (1e-06, 14018.514809196724),
 Text(0.5, 0, 'MSE for avg. E FR (Hz ^ 2)')]

In [132]:
minVal = 1e0

zData = np.arange(MSEExcUnitsHat.size) + 2

f, ax = plt.subplots()
ax.scatter(MSEExcUnitsHat, MSEInhUnitsHat, s=3, c=zData, cmap=plt.cm.viridis, alpha=0.5)
plt.xscale('log')
plt.yscale('log')
ax.set(xlim=(minVal, MSEExcUnitsHat.max()), ylim=(minVal, MSEInhUnitsHat.max()),
       xlabel='MSE for avg. E FR (Hz ^ 2)', ylabel='MSE for avg. I FR (Hz ^ 2)')

[(1.0, 148110.62455567857),
 Text(0, 0.5, 'MSE for avg. I FR (Hz ^ 2)'),
 (1.0, 20629.2448720937),
 Text(0.5, 0, 'MSE for avg. E FR (Hz ^ 2)')]

In [141]:
f, ax = plt.subplots()
ax.plot(frSumExc, color='green')
ax.plot(frSumInh, color='red')
# ax.plot(frSum, color='blue')
plt.yscale('log')
ax.legend(('Exc', 'Inh', 'Exc+Inh'))
ax.set(xlabel='Trial #', ylabel='MovAvg of MSE for avg. FR (Hz ^ 2)')

[Text(0, 0.5, 'MovAvg of MSE for avg. FR (Hz ^ 2)'), Text(0.5, 0, 'Trial #')]

In [152]:
movAvgWidth = 31

f, ax = plt.subplots()
# ax.plot(moving_average(wSum, movAvgWidth))
ax.plot(allAbsdW)
ax.set(xlabel='Trial #', ylabel='Abs. MovAvg. of dW (pA)')

[Text(0, 0.5, 'Abs. MovAvg. of dW (pA)'), Text(0.5, 0, 'Trial #')]

In [151]:
minVal = 1e0

zData = np.arange(frSumExc.size) + 2

f, ax = plt.subplots()
ax.scatter(frSumExc, frSumInh, s=3, c=zData, cmap=plt.cm.viridis, alpha=0.5)
plt.xscale('log')
plt.yscale('log')
ax.set(xlim=(minVal, MSEExcUnitsHat.max()), ylim=(minVal, MSEInhUnitsHat.max()),
       xlabel='MSE for movAvg. E FR (Hz ^ 2)', ylabel='MSE for movAvg. I FR (Hz ^ 2)')

[(1.0, 148110.62455567857),
 Text(0, 0.5, 'MSE for movAvg. I FR (Hz ^ 2)'),
 (1.0, 20629.2448720937),
 Text(0.5, 0, 'MSE for movAvg. E FR (Hz ^ 2)')]

In [93]:
# there is not enough information to calculate mean(abs(dwEE)) but it could be re-calculated like this...

nTrials = R.trialUpFRExcUnits.shape[0]

trialMAdwEE = np.empty((nTrials,))
trialMAdwIE = np.empty((nTrials,))
trialMAdwEI = np.empty((nTrials,))
trialMAdwII = np.empty((nTrials,))

# get some adjacency matrix and nPre
aEE = adjacency_matrix_from_flat_inds(R.p['nExc'], R.p['nExc'], R.preEE, R.posEE)
aEI = adjacency_matrix_from_flat_inds(R.p['nInh'], R.p['nExc'], R.preEI, R.posEI)
aIE = adjacency_matrix_from_flat_inds(R.p['nExc'], R.p['nInh'], R.preIE, R.posIE)
aII = adjacency_matrix_from_flat_inds(R.p['nInh'], R.p['nInh'], R.preII, R.posII)

nIncomingExcOntoEachExc = aEE.sum(0)
nIncomingInhOntoEachExc = aEI.sum(0)
nIncomingExcOntoEachInh = aIE.sum(0)
nIncomingInhOntoEachInh = aII.sum(0)

t0 = datetime.now()

for trialInd in range(nTrials):
    
    movAvgUpFRExcUnits = R.trialUpFRExcUnits[trialInd, :] * Hz
    movAvgUpFRInhUnits = R.trialUpFRInhUnits[trialInd, :] * Hz
    
    movAvgUpFRInhUnitsPreToPostExc = np.matmul(movAvgUpFRInhUnits, aEI) / nIncomingInhOntoEachExc
    movAvgUpFRExcUnitsPreToPostInh = np.matmul(movAvgUpFRExcUnits, aIE) / nIncomingExcOntoEachInh

    movAvgUpFRExcUnits[movAvgUpFRExcUnits < 1 * Hz] = 1 * Hz
    movAvgUpFRInhUnits[movAvgUpFRInhUnits < 1 * Hz] = 1 * Hz

    movAvgUpFRInhUnitsPreToPostExc[movAvgUpFRInhUnitsPreToPostExc > (2 * R.p['setUpFRInh'] - 1 * Hz)] = (2 * R.p[
        'setUpFRInh'] - 1 * Hz)
    movAvgUpFRExcUnitsPreToPostInh[movAvgUpFRExcUnitsPreToPostInh > (2 * R.p['setUpFRExc'] - 1 * Hz)] = (2 * R.p[
        'setUpFRExc'] - 1 * Hz)

    movAvgUpFRExcUnits[movAvgUpFRExcUnits > (2 * R.p['setUpFRExc'] - 1 * Hz)] = (2 * R.p['setUpFRExc'] - 1 * Hz)
    movAvgUpFRInhUnits[movAvgUpFRInhUnits > (2 * R.p['setUpFRInh'] - 1 * Hz)] = (2 * R.p['setUpFRInh'] - 1 * Hz)
    
    if R.p['useRule'] == 'cross-homeo-pre-outer':
        
        dwEE = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits,
                                      (R.p['setUpFRInh'] - movAvgUpFRInhUnitsPreToPostExc)) * Hz
        dwEI = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits,
                                       (R.p['setUpFRInh'] - movAvgUpFRInhUnitsPreToPostExc)) * Hz
        dwIE = -R.p['alpha1'] * np.outer(movAvgUpFRExcUnits,
                                       (R.p['setUpFRExc'] - movAvgUpFRExcUnitsPreToPostInh)) * Hz
        dwII = R.p['alpha1'] * np.outer(movAvgUpFRInhUnits,
                                      (R.p['setUpFRExc'] - movAvgUpFRExcUnitsPreToPostInh)) * Hz

        # save the mean absolute delta in pA
        # dwEE et al MUST be matrices of the right size here
        trialMAdwEE[trialInd] = np.mean(np.fabs(dwEE[R.preEE, R.posEE] / pA))
        trialMAdwEI[trialInd] = np.mean(np.fabs(dwEI[R.preEI, R.posEI] / pA))
        trialMAdwIE[trialInd] = np.mean(np.fabs(dwIE[R.preIE, R.posIE] / pA))
        trialMAdwII[trialInd] = np.mean(np.fabs(dwII[R.preII, R.posII] / pA))

    elif R.p['useRule'] == 'cross-homeo-pre-scalar':

        dwEECH1 = R.p['alpha1'] * movAvgUpFRExcUnits * (R.p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz
        dwEICH1 = -R.p['alpha1'] * movAvgUpFRInhUnits * (R.p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz
        dwIECH1 = -R.p['alpha1'] * movAvgUpFRExcUnits * (R.p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz
        dwIICH1 = R.p['alpha1'] * movAvgUpFRInhUnits * (R.p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz

        dwEE = np.tile(dwEECH1.reshape(-1, 1), R.p['nExc'])
        dwEI = np.tile(dwEICH1.reshape(-1, 1), R.p['nExc'])
        dwIE = np.tile(dwIECH1.reshape(-1, 1), R.p['nInh'])
        dwII = np.tile(dwIICH1.reshape(-1, 1), R.p['nInh'])

        # save the mean absolute delta in pA
        # dwEE et al MUST be matrices of the right size here
        trialMAdwEE[trialInd] = np.mean(np.fabs(dwEE[R.preEE, R.posEE] / pA))
        trialMAdwEI[trialInd] = np.mean(np.fabs(dwEI[R.preEI, R.posEI] / pA))
        trialMAdwIE[trialInd] = np.mean(np.fabs(dwIE[R.preIE, R.posIE] / pA))
        trialMAdwII[trialInd] = np.mean(np.fabs(dwII[R.preII, R.posII] / pA))

    elif R.p['useRule'] == 'cross-homeo-pre-outer-homeo':

        # cross-homeo with presynaptic multiplier (outer product)
        # since outer strips units and because of alpha we multiply by Hz to convert to amps
        dwEECH = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits,
                                        (R.p['setUpFRInh'] - movAvgUpFRInhUnitsPreToPostExc)) * Hz
        dwEICH = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits,
                                         (R.p['setUpFRInh'] - movAvgUpFRInhUnitsPreToPostExc)) * Hz
        dwIECH = -R.p['alpha1'] * np.outer(movAvgUpFRExcUnits,
                                         (R.p['setUpFRExc'] - movAvgUpFRExcUnitsPreToPostInh)) * Hz
        dwIICH = R.p['alpha1'] * np.outer(movAvgUpFRInhUnits,
                                        (R.p['setUpFRExc'] - movAvgUpFRExcUnitsPreToPostInh)) * Hz

        # regular homeo (outer product)
        # since outer strips units and because of alpha we multiply by Hz to convert to amps
        dwEEH = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits, (R.p['setUpFRExc'] - movAvgUpFRExcUnits)) * Hz
        dwEIH = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits, (R.p['setUpFRExc'] - movAvgUpFRExcUnits)) * Hz
        dwIEH = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits, (R.p['setUpFRInh'] - movAvgUpFRInhUnits)) * Hz
        dwIIH = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits, (R.p['setUpFRInh'] - movAvgUpFRInhUnits)) * Hz

        dwEE = dwEECH + dwEEH
        dwEI = dwEICH + dwEIH
        dwIE = dwIECH + dwIEH
        dwII = dwIICH + dwIIH

        # save the mean absolute delta in pA
        # dwEE et al MUST be matrices of the right size here
        trialMAdwEE[trialInd] = np.mean(np.fabs(dwEE[R.preEE, R.posEE] / pA))
        trialMAdwEI[trialInd] = np.mean(np.fabs(dwEI[R.preEI, R.posEI] / pA))
        trialMAdwIE[trialInd] = np.mean(np.fabs(dwIE[R.preIE, R.posIE] / pA))
        trialMAdwII[trialInd] = np.mean(np.fabs(dwII[R.preII, R.posII] / pA))
        
    elif R.p['useRule'] == 'cross-homeo-pre-scalar-homeo':

        dwEECH = R.p['alpha1'] * movAvgUpFRExcUnits * (R.p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz
        dwEICH = -R.p['alpha1'] * movAvgUpFRInhUnits * (R.p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz
        dwIECH = -R.p['alpha1'] * movAvgUpFRExcUnits * (R.p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz
        dwIICH = R.p['alpha1'] * movAvgUpFRInhUnits * (R.p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz

        # regular homeo (outer product)
        # since outer strips units and because of alpha we multiply by Hz to convert to amps
        dwEEH = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits, (R.p['setUpFRExc'] - movAvgUpFRExcUnits)) * Hz
        dwEIH = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits, (R.p['setUpFRExc'] - movAvgUpFRExcUnits)) * Hz
        dwIEH = R.p['alpha1'] * np.outer(movAvgUpFRExcUnits, (R.p['setUpFRInh'] - movAvgUpFRInhUnits)) * Hz
        dwIIH = -R.p['alpha1'] * np.outer(movAvgUpFRInhUnits, (R.p['setUpFRInh'] - movAvgUpFRInhUnits)) * Hz

        dwEE = dwEECH.reshape(-1, 1) + dwEEH
        dwEI = dwEICH.reshape(-1, 1) + dwEIH
        dwIE = dwIECH.reshape(-1, 1) + dwIEH
        dwII = dwIICH.reshape(-1, 1) + dwIIH

        # save the mean absolute delta in pA
        # dwEE et al MUST be matrices of the right size here
        trialMAdwEE[trialInd] = np.mean(np.fabs(dwEE[R.preEE, R.posEE] / pA))
        trialMAdwEI[trialInd] = np.mean(np.fabs(dwEI[R.preEI, R.posEI] / pA))
        trialMAdwIE[trialInd] = np.mean(np.fabs(dwIE[R.preIE, R.posIE] / pA))
        trialMAdwII[trialInd] = np.mean(np.fabs(dwII[R.preII, R.posII] / pA))
        
t1 = datetime.now()
print('took', t1-t0)

took 0:15:31.156521


In [102]:
f, ax = plt.subplots()
ax.plot(trialMAdwEE)

In [5]:
# check something about each file...

for seedInd in range(9):
    targetFile1, targetFile2 = analysisDict[seedInd]
    R1 = Results()
    try:
        R1.init_from_file(targetFile1, targetPath)
        print(R1.p['tauUpFRTrials'])
    except FileNotFoundError:
        print('one of the files wasn''t found for some reason')
    R2 = Results()
    try:
        R2.init_from_file(targetFile2, targetPath)
        print(R2.p['tauUpFRTrials'])
    except FileNotFoundError:
        print('one of the files wasn''t found for some reason')

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [20]:
# script to create all the figures for the multiTraining sessions

for seedInd in range(9):
    targetFile1, targetFile2 = analysisDict[seedInd]
    R1 = Results()
    R1.init_from_file(targetFile1, targetPath)
    R2 = Results()
    R2.init_from_file(targetFile2, targetPath)
    R = combine_two_sessions(R1, R2)

    print(R.rID)
    print(R.p['setUpFRExc'], R.p['tauUpFRTrials'], R.p['alpha1'], R.p['spikeInputAmplitude'])
    plt.close('all')
    pdfObject = PdfPages(targetPath + 'reports/' + R.rID + '_report.pdf')
    removeOutliers = False

    generate_description(R)

    FR_weights(R)
    plt.savefig(pdfObject, format='pdf')

    # firstConvergentInd = int(R.trialwEE.size / 2)
    # firstConvergentInd = 4700
    firstConvergentInd = determine_first_convergent_trial(R)
    # firstConvergentInd = determine_first_convergent_trial(R) + 1500
    print('We think the first convergent index is', firstConvergentInd)

    determine_drift_rates(R, firstConvergentInd)
    print(R.driftRateEE)
    print(R.driftRateEI)
    print(R.driftRateIE)
    print(R.driftRateII)

    # convergence process
    FR_scatter(R, endTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')
    FR_hist1d_compare(R, endTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')
    FR_weights_std(R, endTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')
    weights_scatter_matrix(R, endTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')

    # post convergence
    FR_scatter(R, startTrialInd=firstConvergentInd, removeOutliers=removeOutliers)
    plt.savefig(pdfObject, format='pdf')
    FR_hist2d(R, startTrialInd=firstConvergentInd, removeOutliers=removeOutliers)
    plt.savefig(pdfObject, format='pdf')
    FR_hist1d_compare(R, startTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')
    FR_weights_std(R, startTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')
    weights_scatter_matrix(R, startTrialInd=firstConvergentInd)
    plt.savefig(pdfObject, format='pdf')

    # epilogue
    weights_hist1d_compare(R)
    plt.savefig(pdfObject, format='pdf')
    weights_matrix_compare(R)
    plt.savefig(pdfObject, format='pdf')

    pdfObject.close()

classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_seed0__2021-08-21-09-33
5. Hz 1 8. fC 0.98
We think the first convergent index is 1164
0.3438049879554385
-0.25267338753683105
0.1026854251972901
-0.7582299550928737


WARNING    ..\stats.py:17: RuntimeWarning: invalid value encountered in greater
  a_outlying = np.logical_or(a > a_mean + sd_thresh * a_std, a < a_mean - sd_thresh * a_std)
 [py.warnings]
WARNING    ..\stats.py:17: RuntimeWarning: invalid value encountered in less
  a_outlying = np.logical_or(a > a_mean + sd_thresh * a_std, a < a_mean - sd_thresh * a_std)
 [py.warnings]
WARNING    ..\analysis.py:320: RuntimeWarning: invalid value encountered in less
  wFullInitPlot[wFullInitPlot < vlims[0]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:321: RuntimeWarning: invalid value encountered in greater
  wFullInitPlot[wFullInitPlot > vlims[1]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:323: RuntimeWarning: invalid value encountered in less
  wFullFinalPlot[wFullFinalPlot < vlims[0]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:324: RuntimeWarning: invalid value encountered in greater
  wFullFinalPlot[wFullFinalPlot > vlims[1]] = np.nan
 [py.warnings]
WARNING    C:\Users\mikejseay

classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_seed1__2021-08-21-09-34
5. Hz 1 8. fC 0.98
We think the first convergent index is 536
1.9519028104149363
-0.05276630252854969
1.3321280892914467
-2.0132216651786075


WARNING    ..\stats.py:17: RuntimeWarning: invalid value encountered in greater
  a_outlying = np.logical_or(a > a_mean + sd_thresh * a_std, a < a_mean - sd_thresh * a_std)
 [py.warnings]
WARNING    ..\stats.py:17: RuntimeWarning: invalid value encountered in less
  a_outlying = np.logical_or(a > a_mean + sd_thresh * a_std, a < a_mean - sd_thresh * a_std)
 [py.warnings]
WARNING    ..\analysis.py:320: RuntimeWarning: invalid value encountered in less
  wFullInitPlot[wFullInitPlot < vlims[0]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:321: RuntimeWarning: invalid value encountered in greater
  wFullInitPlot[wFullInitPlot > vlims[1]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:323: RuntimeWarning: invalid value encountered in less
  wFullFinalPlot[wFullFinalPlot < vlims[0]] = np.nan
 [py.warnings]
WARNING    ..\analysis.py:324: RuntimeWarning: invalid value encountered in greater
  wFullFinalPlot[wFullFinalPlot > vlims[1]] = np.nan
 [py.warnings]
WARNING    C:\Users\mikejseay

classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_seed2__2021-08-21-09-34
5. Hz 1 8. fC 0.98
We think the first convergent index is 5626
1.7345976444595907
0.7466571490226588
1.018813379420174
-1.0040977452787898


KeyboardInterrupt: 

In [34]:
overallDF.columns = overallDF.sort_index(axis=1, level=[0, 1], ascending=[True, True]).columns